<a href="https://colab.research.google.com/github/nguyen-nhat-mai/methane-leak-detection/blob/main/Web_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build web app for classifier model

In [99]:
!pip install streamlit -q

In [100]:
!pip install streamlit_folium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [103]:
# Load the airquality first to avoid web app lag
metadata=pd.read_csv("metadata.csv")
import requests
import json
# Define the base API URL
api_url = 'https://air-quality-api.open-meteo.com/v1/air-quality'

# Create empty lists to store the API responses
api_responses = []

# Iterate over the rows of the DataFrame
for index, row in metadata.iterrows():
    # Get the latitude and longitude values from the current row
    latitude = row['lat']
    longitude = row['lon']
    
    # Create the API URL with the latitude and longitude parameters
    url = f'{api_url}?latitude={latitude}&longitude={longitude}&hourly=european_aqi&start_date=2022-08-01&end_date=2023-01-31'
    
    # Send the API request
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == requests.codes.ok:
        # Append the API response to the list
        api_responses.append(np.mean([value for value in json.loads(response.text)["hourly"]["european_aqi"] if value is not None]))
    else:
        # Handle the error if the request was not successful
        print(f"Error for latitude {latitude}, longitude {longitude}: {response.status_code}, {response.text}")
metadata['api_response'] = api_responses
metadata.head()

,date,id_coord,plume,set,lat,lon,coord_x,coord_y,path,api_response
0,20230223,id_6675,yes,train,31.528750,74.330625,24,47,images/plume/20230223_methane_mixing_ratio_id_...,117.434028
1,20230103,id_2542,yes,train,35.538000,112.524000,42,37,images/plume/20230103_methane_mixing_ratio_id_...,70.871991
2,20230301,id_6546,yes,train,21.060000,84.936667,58,15,images/plume/20230301_methane_mixing_ratio_id_...,68.038426
3,20230225,id_6084,yes,train,26.756667,80.973333,28,62,images/plume/20230225_methane_mixing_ratio_id_...,99.746296
4,20230105,id_2012,yes,train,34.800000,40.770000,59,44,images/plume/20230105_methane_mixing_ratio_id_...,35.661111


In [106]:
metadata.to_csv('metadata_aq.csv', index=False)

Reference:

https://towardsdatascience.com/create-an-image-classification-web-app-using-pytorch-and-streamlit-f043ddf00c24#2b4c
https://www.youtube.com/watch?v=NEhrkeF2o_M

In [109]:
# Write the web app
%%writefile app.py
import streamlit as st
from sklearn.preprocessing import MinMaxScaler
from PIL import Image
import time
import pandas as pd
import numpy as np
import cv2
import tensorflow as tf
import base64
from io import BytesIO
import altair as alt
import folium
from streamlit_folium import st_folium, folium_static
import requests
import json

#---------------------- MODEL PREDICTION-----------------------#
   
def preprocess_image(img):
    img = img.resize((64, 64), Image.ANTIALIAS)  # Resize the image
    data = np.array(img)
    data = np.expand_dims(data, axis=-1)
    data = data.astype('float32') / 255.0
    return np.array([data])

def preprocess_location(image_name,df_normalized):
    matching_row = df_normalized[df_normalized['path'].str.contains(image_name)]
    lat = matching_row['lat'].values[0]
    api_response = matching_row['api_response'].values[0]
    lon = matching_row['lon'].values[0]
    coord_x = matching_row['coord_x'].values[0]
    coord_y = matching_row['coord_y'].values[0]
    return np.array([[lat, api_response, lon, coord_x, coord_y]])

def predict(image, image_name, df_normalized):
    # Load model
    model = tf.keras.models.load_model("model_v2.h5")
    # Load data
    loc_data = preprocess_location(image_name,df_normalized)
    img_data = preprocess_image(image)
    # Do inference
    y_pred = model.predict({"location": loc_data, "img": img_data})
    y_prob = y_pred.flatten()*100

    max_idx = np.argmax(y_prob)
    max_prob = y_prob[max_idx]

    y_class = (y_pred >= 0.5).astype(int).flatten()
    max_class = y_class[max_idx]

    # Return the class and corresponding probability
    return [(max_class, max_prob)]

# ------------------------------WEB APP----------------------------------#

# Define a function to create a download link for a given DataFrame
def download_link(df, filename, text):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode('utf-8')).decode()  # base64 encoding
    href = f"data:text/csv;base64,{b64}"
    return f'<a href="{href}" download="{filename}">{text}</a>'

# Get dictionary of geolocation for id_coord
geo_map = pd.read_csv("metadata_aq.csv")
geo_dict = {row['id_coord']: (row['lat'], row['lon']) for _, row in geo_map.iterrows()}

# Get the normalized location
scaler = MinMaxScaler()
scaled_values = scaler.fit_transform(geo_map[['lat', 'lon']])
df_normalized = geo_map.copy()
df_normalized[['lat', 'lon']] = scaled_values

# Start the display on web app
st.set_option('deprecation.showfileUploaderEncoding', False)
st.title("CleanR - Detect Methane Leaks")
st.write("")
st.sidebar.header("Menu")
tabs = ["Prediction on uploaded images", "Leak detection map", "Leak track over time by id_coord", "Prediction detail"]
selected_tab = st.sidebar.radio("", tabs)


if selected_tab == "Prediction on uploaded images":
    file_up = st.file_uploader("Upload your images", type=["jpg", "jpeg", "png","tif", "tiff"], accept_multiple_files=True)
    if file_up is not None:
        st.subheader("Prediction on uploaded images")
        all_predictions = pd.DataFrame() # create an empty DataFrame to store all predictions
        for img_file in file_up:
            image = Image.open(img_file)
            image_8bit = image.convert("L")
            st.image(image_8bit, caption=img_file.name, use_column_width=True)
            st.write("")
            with st.spinner('Predicting...'):
                # Get prediction result
                predictions_df = predict(image_8bit,img_file.name.split(".")[0], df_normalized)
                # Append the current predictions to the DataFrame along with the id_coord, date and geolocation
                id_coord = img_file.name.split(".")[0][-7:]
                date = img_file.name.split("_")[0]
                latitude = geo_dict[id_coord][0]
                longitude = geo_dict[id_coord][1]
                predictions_df = [(id_coord,date,latitude,longitude,)+predictions_df[0]]
                all_predictions = all_predictions.append(predictions_df, ignore_index=True)
                # Print out the prediction labels with probability
                st.write("Prediction:", predictions_df[0][4], "-   Probability (%): ", round(predictions_df[0][5],1))
        if not all_predictions.empty:
            all_predictions.columns = ['Id_coord', "Date","Latitude","Longitude",'Label', 'Probability (%)']
            all_predictions.to_csv("predictions.csv", index=False)

if selected_tab == "Leak detection map":
    st.subheader("Leak detection map")
    all_predictions = pd.read_csv("predictions.csv")
    all_predictions['Date'] = pd.to_datetime(all_predictions['Date'], format='%Y%m%d')
    # Create base map that will be centered around the mean latitude and longitude 
    m = folium.Map(location=[all_predictions.Latitude.mean(), all_predictions.Longitude.mean()], 
                 zoom_start=3, control_scale=True)
    # Loop through each row in the dataframe
    for i,row in all_predictions.iterrows():
        # Setup the content of the popup
        iframe = folium.IFrame('id:' + str(row["Id_coord"]))
        # Initialise the popup using the iframe
        popup = folium.Popup(iframe, min_width=300, max_width=300)
        # Add each row to the map
        folium.Marker(location=[row['Latitude'],row['Longitude']],
                  popup = popup, c=row['Id_coord'],
                  icon=folium.Icon(color='red',icon='info-sign')).add_to(m)
    st_data = st_folium(m, width=700)

if selected_tab == "Leak track over time by id_coord":
    st.subheader("Leak track over time by id_coord")
    all_predictions = pd.read_csv("predictions.csv")
    all_predictions['Date'] = pd.to_datetime(all_predictions['Date'], format='%Y%m%d')
    id_coord_list = list(all_predictions['Id_coord'].unique())
    selected_id_coord = st.sidebar.selectbox("Select an id_coord:", id_coord_list)
    filtered_predictions = all_predictions.loc[all_predictions['Id_coord'] == selected_id_coord]
    chart = alt.Chart(filtered_predictions).mark_line(color="#ff2b2b").encode(x='Date',y='Probability (%)')
    st.altair_chart(chart, use_container_width=True)

if selected_tab == "Prediction detail":
    st.subheader("Prediction Detail")
    all_predictions = pd.read_csv("predictions.csv")
    all_predictions['Date'] = pd.to_datetime(all_predictions['Date'], format='%Y%m%d')
    st.markdown(download_link(all_predictions, "prediction.csv", "Download CSV"), unsafe_allow_html=True)
    st.write("")
    st.write(all_predictions)

Overwriting app.py


In [110]:
# Run app.py and made available on a local URL
!streamlit run app.py & npx localtunnel --port 8501

npx: installed 22 in 2.149s



  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.86.49.100:8501

your url is: https://thick-teams-take.loca.lt
2023-05-15 13:42:27.843266: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-15 13:42:30.564 `label` got an empty value. This is discouraged for accessibility reasons and may be disallowed in the future by raising an exception. Please provide a non-empty label and hide it with label_visibility if needed.
2023-05-15 13:42:37.938 `label` got an empty value. This is discouraged for accessibility reasons and may be disallowed in the future by raising an exception. Please provide a non-empty label and hide it with label_visibility if needed.
1/1 [==============================] - 0s 125ms/step
/content/app.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pand